# Création d'un dictionnaire à partir de fichiers csv

In [1]:
import csv
from datetime import datetime
import os
import glob

def create_timestamp(date):
    dt = datetime(int(date[0:4]), int(date[5:7]), int(date[8:10]), 
                   int(date[11:13]), int(date[14:16]), int(date[17:19]))
    
    dt = datetime.timestamp(dt)
    return dt


def filename_from_path(file):
    
    begin = len(file)-1
    
    while file[begin] != ".":
        begin -= 1
        
    end = begin
    
    while file[begin] != "\\":
          begin -= 1
          
    begin += 1
    return file[begin:end]
    
    

notebook_path = os.path.abspath("Notebook.ipynb")
chemin = os.path.join(os.path.dirname(notebook_path), "results/assets")

paths = glob.glob(chemin+'/*')

data = []

for i in range(0, 100):
    
    name = filename_from_path(paths[i])
    
    with open(paths[i], newline='') as csvfile:
        reader = csv.DictReader(csvfile)

        for row in reader:
            point = {}      
            point['asset_id'] = name
            point['recorded_at'] = create_timestamp(row['recorded_at'])        
            
            g = {}
            coord = []
            coord.append(float(row['longitude']))
            coord.append(float(row['latitude']))
            g['coordinates'] = coord
            g['type'] = "Point"
            
            loc = {}
            loc['geo'] = g
            point['location'] = loc
            point['moving'] = 1  # en mouvement par défaut
            data.append(point)
        
    data = sorted(data, key=lambda point: point['recorded_at'])   # tri selon recorded_at

    
print('done')

done


# Importation du dictionnaire dans la base de données 'fcd'

In [2]:
import pymongo

client = pymongo.MongoClient('localhost',27017)
mydb = client["fcd"]
collection = mydb["assets"]
print('insert')
collection.insert_many(data)
print('done')

insert
done


# Créer une collection pour les résumés

In [9]:
import pymongo
client = pymongo.MongoClient('localhost',27017)
mydb = client["fcd"]
collection_resume = mydb["resume"]
data=[]

resume = {} 
resume['name_resume'] = "Requete1"
resume['density'] = None
resume['neighbours_id'] = None
resume['period'] = None
z = {}
z['center'] = None
z['radius'] = None
resume['zone'] = z
resume['settings'] = None
data.append(resume)
print('insert')
collection_resume.insert_many(data)
print('done')

insert
done


# Ajout de l'index géo spatial

In [3]:
from pymongo import GEOSPHERE

client = pymongo.MongoClient('localhost')
collection = client.fcd.assets

resp = collection.create_index([("location.geo", GEOSPHERE)])
print ("index response:", resp)

index response: location.geo_2dsphere


# Application de l'algorithme des stay points

In [4]:
import math

# in minutes
def time_gap(current, new):
    
    td = abs(current - new)
    res = td/60
    return res

# in meters
def distance(lat1, lat2, lon1, lon2):
    R = 6372800  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    d = 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return d


def algorithm_stayPoint_detection(P, distThreh, timeThreh):

    i = 0
    pointNum = len(P)
    # print('taille', pointNum)
    SP = []

    while i < pointNum-1:
        
        j = i + 1
        
        while j < pointNum:
            dist = distance(float(P[i]['location']['geo']['coordinates'][1]),
                            float(P[j]['location']['geo']['coordinates'][1]),
                            float(P[i]['location']['geo']['coordinates'][0]),
                            float(P[j]['location']['geo']['coordinates'][0]))

            if dist > distThreh:
                break
                
            j+=1
        
        j-=1
        
        deltaT = time_gap(P[i]['recorded_at'], P[j]['recorded_at'])
        if deltaT > timeThreh:
            S = {}
            S['arv'] = i
            S['lev'] = j
            SP.append(S)
            i=j
        else:
            i+=1

            
    return SP


#SP = algorithm_stayPoint_detection(data, 200, 30)
print('done')

done


# Modification de l'attribut moving dans 'fcd'

In [5]:

def set_stayPoints_database(SP, asset_points):

    for stayP in SP:

        for i in range(stayP['arv']+1, stayP['lev']):

            filtre = {'_id': asset_points[i]['_id']}
            collection.update_one(filtre, {"$set": {"moving": 0}})
            
            

# tri par asset
data = sorted(data, key=lambda point: point['asset_id'])
current_id = data[0]['asset_id']

asset_points = []
count_SP = 0

for point in data:

    if point['asset_id'] == current_id:
        asset_points.append(point)
        
    else:
        # tri chronologique           
        asset_points = sorted(asset_points, key=lambda point: point['recorded_at'])
        # detection des stay points
        SP = algorithm_stayPoint_detection(asset_points, 200, 30)
        count_SP += len(SP)
        
        # mofication de la base de données
        set_stayPoints_database(SP, asset_points)
        
        #reinitialisation
        current_id = point['asset_id']
        asset_points = []
        asset_points.append(point)
    
    
    
# pour le dernier asset

# tri chronologique           
asset_points = sorted(asset_points, key=lambda point: point['recorded_at'])
# detection des stay points
SP = algorithm_stayPoint_detection(asset_points, 200, 30)
count_SP += len(SP)
# mofication de la base de données
set_stayPoints_database(SP, asset_points)

print(count_SP, 'stay points detected')
print('done')

1730 stay points detected
done
